# Download YouTube Video's Audio

In [ ]:
! pip install pytube -q 

     |████████████████████████████████| 56 kB 2.7 MB/s 


In [ ]:
from pytube import YouTube

In [ ]:
#VIDEO_URL = "https://www.youtube.com/watch?v=hWLf6JFbZoo" #obama

In [ ]:
VIDEO_URL = 'https://www.youtube.com/watch?v=h-JVjs9AAmQ' #batman

In [ ]:
#VIDEO_URL = 'https://youtu.be/qNJRGHk7sN8'

In [ ]:
yt = YouTube(VIDEO_URL)

In [ ]:
yt.streams \
  .filter(only_audio = True, file_extension = 'mp4') \
  .first() \
  .download(filename = 'ytaudio.mp4') \

'/content/ytaudio.mp4'

In [ ]:
! ffmpeg -i ytaudio.mp4 -acodec pcm_s16le -ar 16000 ytaudio.wav

ffmpeg version 3.4.11-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-li

# English ASR with HuggingSound

In [ ]:
!pip install huggingsound -q 

     |████████████████████████████████| 312 kB 16.9 MB/s 
     |████████████████████████████████| 4.7 MB 63.5 MB/s 
     |████████████████████████████████| 25.3 MB 1.2 MB/s 
     |████████████████████████████████| 3.4 MB 52.6 MB/s 
     |████████████████████████████████| 120 kB 61.2 MB/s 
     |████████████████████████████████| 212 kB 68.5 MB/s 
     |████████████████████████████████| 115 kB 70.5 MB/s 
     |████████████████████████████████| 50 kB 6.3 MB/s 
     |████████████████████████████████| 127 kB 70.6 MB/s 
     |████████████████████████████████| 6.6 MB 57.7 MB/s 


In [ ]:
from huggingsound import SpeechRecognitionModel


In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
device

'cuda'

In [ ]:
model = SpeechRecognitionModel("jonatasgrosman/wav2vec2-large-xlsr-53-english", device = device)


INFO:huggingsound.speech_recognition.model:Loading model...


OUT OF MEMORY (OOM) error

# Audio Chunking

In [ ]:
import librosa

In [ ]:
input_file = '/content/ytaudio.wav'

In [ ]:
print(librosa.get_samplerate(input_file))

# Stream over 30 seconds chunks rather than load the full file
stream = librosa.stream(
    input_file,
    block_length=30,
    frame_length=16000,
    hop_length=16000
)

16000


In [ ]:
import soundfile as sf


In [ ]:
for i,speech in enumerate(stream):
  sf.write(f'{i}.wav', speech, 16000)

In [ ]:
i

8

# Audio Transcription / ASR / Speech to Text

In [ ]:
audio_path =[]
for a in range(i+1):
  audio_path.append(f'/content/{a}.wav') 

In [ ]:
audio_path

['/content/0.wav',
 '/content/1.wav',
 '/content/2.wav',
 '/content/3.wav',
 '/content/4.wav',
 '/content/5.wav',
 '/content/6.wav',
 '/content/7.wav',
 '/content/8.wav']

In [ ]:
transcriptions = model.transcribe(audio_path)

100%|██████████| 9/9 [00:11<00:00,  1.24s/it]


In [ ]:
full_transcript = ' '

In [ ]:
for item in transcriptions:
  full_transcript += ''.join(item['transcription'])

In [ ]:
len(full_transcript)

3091

# Text Summarization 

In [ ]:
from transformers import pipeline

In [ ]:
summarization = pipeline('summarization')

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
summarized_text = summarization(full_transcript)

In [ ]:
summarized_text[0]['summary_text']

" The role of cat woman has been played by litedary access arears from rto kit and shell fifer . Batman is about etwo sides of drauma batman's born of trama andtis film maybes the ridler and tat is kind of the seed from which everything else grew ."

Text Chunking before Summarization

In [ ]:
num_iters = int(len(full_transcript)/1000)
summarized_text = []
for i in range(0, num_iters + 1):
  start = 0
  start = i * 1000
  end = (i + 1) * 1000
  #print("input text \n" + full_transcript[start:end])
  out = summarization(full_transcript[start:end], min_length = 5, max_length=20)
  out = out[0]
  out = out['summary_text']
 # print("Summarized text\n"+out)
  summarized_text.append(out)

#print(summarized_text)